## Introducing the Community Earth System Model (CESM)
This notebook is adapted from [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook) by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

## About the CESM

### Overview

The CESM is one example of complex coupled GCMS utilized by the Intergovernmental Panel for Climate Change report process.  It is developed and maintained at NCAR by a group of software engineers and climate processes.  The code is open-source, with new pieces contributed by a wide variety of users.

### Key Componenets

The CESM is a modular piece of software, meaning that a researcher can use which submodels to combine together to best answer their invididual research question with the computer power they have available.

Pieces of the CSEM include:
- Atmospheric model (AGCM): Community Atmosphere Model (CAM)
    -- This model maintains a resolution of approximately 2º lat/long, meaning that while it resolves synoptic-scale dynamics like storm tracks and cyclones, it is not capable of resolving mesoscale and smaller events like thunderstorms or clouds.  The model conserves momentum, mass, energy, water, and the equation of state, while parameterizing weather phenomena and solving equations of radiative transfer based on the composition of the atmosphere, absorption properties of different gases, and the radiative effects of clouds
- Ocean model (OGCM): Parallel Ocean Program (POP)
    -- This model has a 1º lat/long resolution and models the exchanges of heat, water, and momentum in the atmosphere and sea ice.  It includes a full 3D simulation of the currents.
- Land surface model: Community Land Model (CLM)
    -- Like the atmospheric model, this model maintains a 2º lat/long resolution.  It's complex, but, in short, determined the surface fluxes of heat, water, and momentum based on vegetation types.
- Sea ice model: Community Ice CodE (CICE)
    -- This model has a 1º lat/long resolution.  It is incredibly complex and uses thermodynamics to determine freezing and melting and momentum equations to track ice motion and deformation.

### Setting up the model
In order to run the model, we need to give it realistic atmospheric compositions, solar radiations, et cetera.  We will first perform a control - a preindustrial run - and get the average of several years, then do our test run - with double the original atmospheric CO2 - and run it several times to allow the model to adjust to the new equilibrium.

All of our data is stored in `NetCDF` files, which contain self-describing gridded data 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

### Boundary conditions: continents and topography

To begin setting up our models, we are first going to input the topography file.  This file was retrieved from the [class Slack page](https://app.slack.com/client/T01JVA6NV46/C01JVA6PJCE) and saved to each of our computers, meaning that the user of this lab write-up will need to update the below file path to retrieve the file.

In [ ]:
#  Open the topography dataset
#  Remember to update file path to the location of your file
topo = xr.open_dataset( "/Users/Andrea/Downloads/USGS-gtopo30_1.9x2.5_remap_c050602.nc" )
print(topo)

# access individual variables within the 'xarray.Dataset'
topo.PHIS

### Plotting the topography
After investigating the topography dataset, we are ready to plot the topography of the Earth's surface on this 2º lat/long grid based on the geopotential variable.  

The below code makes a colorful topographic plot, with a land-scea mask so that nothing is plotted on ocean-covered plot points.  

In [ ]:
g = 9.8  # gravity in m/s2
meters_per_kilometer = 1E3 # define that there are 1000 meters per kilometer
height = topo.PHIS / g / meters_per_kilometer  # convert the topography from meters to kilometers
#  Note that we have just created a new xarray.DataArray object that preserves the axis labels
#  Let's go ahead and give it some useful metadata:
height.attrs['units'] = 'km'
height.name = 'height'
height # open xarray.Dataarray height variable
height.plot() # produce topography plot

To produce a more attractive plot, we can use matplotlib.

In [ ]:
#  A filled contour plot of topography with contours every 500 m
lev = np.arange(0., 6., 0.5)
fig1, ax1 = plt.subplots(figsize=(8,4))
# Here we are masking the data to exclude points where the land fraction is zero (water only)
cax1 = ax1.contourf( height.lon, height.lat, 
                    height.where(topo.LANDFRAC>0), levels=lev)
ax1.set_title('Topography (km) and land-sea mask in CESM')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
cbar1 = fig1.colorbar(cax1)

The 2º of our topography plots means that while we can see certain smaller features like Pacific island, these are lumped together into larger plots with fractional land covery for each grid point.

In order to see areas with only "some" water, we can plot the land-sea mask itself.  Areas with 0.0 ocean mask are completely water (like the middle of an ocean) while areas with 1.0 ocean mask are dry land, like the middle of a contient.  Notice that major likes, like Lake Victoria or Lake Eerie, are not visible on this map.

In [ ]:
fig2, ax2 = plt.subplots() # produce a second figure
cax2 = ax2.pcolormesh( topo.lon, topo.lat, topo.LANDFRAC ) #plot longitude on the x-axis, latitude on the y-axis, and a colorscale
ax2.set_title('Ocean mask in CESM') # title 
ax2.set_xlabel('Longitude'); ax2.set_ylabel('Latitude') # x and y axis labels
cbar2 = fig2.colorbar(cax2); # color key

However, notice that the latitude-longitude array, as plotted above, does not have a map projection and is therefore highly distorted.

### Ocean Boundary Conditions

Next, we will visualize the ocean and its interaction with the atmosphere.  Remember that the ocean and sea ice models use a different grid (1º lat/long) than the atmosphere grid (2º lat/long).

First, we will open another dataset.  As before, this dataset is from the class Slack page, and the reader of this notebook should take care to replace the file path with the one for their computer.

In [ ]:
som_input = xr.open_dataset("/Users/Andrea/Downloads/pop_frc.1x1d.090130.nc")
print(som_input)

After inputting the dataset, we determined the mean heat flux out of the ocean.  This information is stored in the 'qdp' field of the above dataset.  By convention, if qdp is greater than zero, heat is going into the ocean; if qdp is less than zero, heat is coming out of the ocean.  We changed the sign in order to plot heat heading out of the ocean and into the atmosphere. 

The data is in the units of Wm-2 and there are 12 x 180 x 360 data points, meaning one 180 x 360 grid for each calendar month.

With this information, we can calculate the average annual heat flux out of the ocean over the year at every grid cell.  

In [ ]:
som_input.qdp # look over the data
(-som_input.qdp.mean(dim='time')).plot() # calculate and graph the mean year-round heat flux out of the ocean

We can then prettied up this graph by adding outlines to the continents, a discrete color scale, and fixing up the labels. 

In [ ]:
#  We can always set a non-standard size for our figure window
fig3, ax3 = plt.subplots(figsize=(10, 6))
lev = np.arange(-700., 750., 50.)
cax3 = ax3.contourf(som_input.xc, som_input.yc, 
                    -som_input.qdp.mean(dim='time'), 
                    levels=lev, cmap=plt.cm.bwr)
cbar3 = fig3.colorbar(cax3)
ax3.set_title( 'CESM: Prescribed heat flux out of ocean (W m$^{-2}$), annual mean', 
              fontsize=14 )
ax3.set_xlabel('Longitude', fontsize=14)
ax3.set_ylabel('Latitude', fontsize=14)
ax3.text(65, 50, 'Annual', fontsize=16 )
ax3.contour(topo.lon, topo.lat, topo.LANDFRAC, levels=[0.5], colors='k');

From this graph, we noticed that most of the incoming heat (blue) was at the equator, especially in the Pacific.  Areas with outgoing heat (red), meanwhile, where around the mid-Atlantic coats around Asia, North America, and the northernmost parts of the Atlantic.

This structure is a result of ocean currents, though we didn't model the ocean currents directly here, but rather described the heat flux patterns as an input to the atmosphere without explaining their underlying structure.

Next, we looked at the data from just one month

In [ ]:
# select by month index (0 through 11)
som_input.qdp.isel(time=0)

#  select by array slicing (but for this you have to know the axis order!)
# by specificying [0,:,:] after the variable name, we selected just January.
som_input.qdp[0,:,:]

After selecting just January, we can plot that data.

In [ ]:
fig4, ax4 = plt.subplots(figsize=(10,4)) 
cax4 = ax4.contourf( som_input.xc, som_input.yc, 
                    -som_input.qdp.isel(time=0), 
                      levels=lev, cmap=plt.cm.bwr)
cbar4 = plt.colorbar(cax4)
ax4.set_title( 'CESM: Prescribed heat flux out of ocean (W m$^{-2}$)', 
              fontsize=14 )
ax3.set_xlabel('Longitude', fontsize=14)
ax3.set_ylabel('Latitude', fontsize=14)
ax4.text(65, 50, 'January', fontsize=12 );
ax4.contour(topo.lon, topo.lat, topo.LANDFRAC, levels=[0.5], colors='k');